In [ ]:
# Example: My preferred approach
import os
# Settings -> Developer Settings -> Personal Access Tokens -> Token (classic)
os.environ['GITHUB_TOKEN'] = ""

GITHUB_USERNAME = "Codfishz"
REPO_NAME       = "ASR"
TOKEN = os.environ.get("GITHUB_TOKEN")
repo_url        = f"https://{TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"
!git clone {repo_url}

Cloning into 'ASR'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 49 (delta 20), reused 38 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 10.82 MiB | 14.90 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [2]:
!cd {REPO_NAME} && git pull

Already up to date.


In [3]:
import os
os.chdir('ASR/Script')

In [4]:
%run "Datapreparation_YOLO.ipynb"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.2
    Uninstalling kaggle-1.7.4.2:
      Successfully uninstalled kaggle-1.7.4.2
Dataset URL: https://www.kaggle.com/datasets/nirmalsankalana/crop-pest-and-disease-detection
License(s): CC0-1.0
✅ Total images found in dataset: 25220
Images before filter: 25220


Saving train: 100%|██████████| 20176/20176 [00:32<00:00, 612.76it/s]



train split summary:
  Total images: 20103



Saving val: 100%|██████████| 2522/2522 [00:04<00:00, 569.98it/s]



val split summary:
  Total images: 2511



Saving test: 100%|██████████| 2522/2522 [00:04<00:00, 615.82it/s]


test split summary:
  Total images: 2512

  Images after filter : 25126



In [5]:
!pip install ultralytics
!pip install torchinfo

import ultralytics
ultralytics.checks()

Ultralytics 8.3.108 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 43.7/235.7 GB disk)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from ultralytics import YOLO
print("GPU available:", torch.cuda.is_available())

GPU available: True


# Active YOLO functions

In [8]:
def setup_active_dataset():

    orig_train_dir = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR)
    active_train_dir = os.path.join(ACTIVE_DATASET_DIR, TRAIN_SUBDIR)
    os.makedirs(active_train_dir, exist_ok=True)

    for cls in os.listdir(orig_train_dir):
        cls_path = os.path.join(orig_train_dir, cls)
        if os.path.isdir(cls_path):
            os.makedirs(os.path.join(active_train_dir, cls), exist_ok=True)
    print(f"Complete creating train_dir {active_train_dir}")

    orig_val_dir = os.path.join(ORIGINAL_DATASET_DIR, VAL_SUBDIR)
    active_val_dir = os.path.join(ACTIVE_DATASET_DIR, VAL_SUBDIR)
    if os.path.exists(orig_val_dir):
        if os.path.exists(active_val_dir):
            shutil.rmtree(active_val_dir)
        shutil.copytree(orig_val_dir, active_val_dir)

    orig_test_dir = os.path.join(ORIGINAL_DATASET_DIR, TEST_SUBDIR)
    active_test_dir = os.path.join(ACTIVE_DATASET_DIR, TEST_SUBDIR)
    if os.path.exists(orig_test_dir):
        if os.path.exists(active_test_dir):
            shutil.rmtree(active_test_dir)
        shutil.copytree(orig_test_dir, active_test_dir)

In [9]:
def get_all_samples():

    samples = []
    orig_train_dir = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR)
    for cls in os.listdir(orig_train_dir):
        cls_path = os.path.join(orig_train_dir, cls)
        if os.path.isdir(cls_path):
            image_files = [f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))]
            for f in image_files:
                samples.append((cls, f))
    return samples

In [10]:
import shutil
def copy_samples(sample_list):

    for cls, file_name in sample_list:
        src_path = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR, cls, file_name)
        dst_path = os.path.join(ACTIVE_DATASET_DIR, TRAIN_SUBDIR, cls, file_name)
        if not os.path.exists(dst_path):
            shutil.copy(src_path, dst_path)

In [11]:
import random
def stratified_sample(all_samples, n_initial):

    samples_by_class = {}
    for cls, filename in all_samples:
        samples_by_class.setdefault(cls, []).append((cls, filename))

    stratified = []
    for cls, samples in samples_by_class.items():
        stratified.append(random.choice(samples))
    remaining_count = n_initial - len(stratified)
    if remaining_count > 0:
        remaining_samples = list(set(all_samples) - set(stratified))
        additional_samples = random.sample(remaining_samples, remaining_count)
        stratified.extend(additional_samples)

    return stratified



# Hyperparameters

In [ ]:
ORIGINAL_DATASET_DIR = '/content/data_yolo'
ACTIVE_DATASET_DIR = '/content/data_active'

TRAIN_SUBDIR = 'train'
VAL_SUBDIR = 'val'
TEST_SUBDIR = 'test'
Num_Train = 20109
N_INITIAL = int(0.3 * Num_Train)
N_PER_PHASE = int(0.1 * Num_Train)
NUM_PHASES = 5

# MC-Dropout

In [ ]:
import torch
import torch.nn as nn

def mc_dropout_inference(model, image, num_passes=20):
    """
    Perform MC Dropout inference and return mean + variance across predictions.
    """
    model.train()
    for module in model.modules():
        if isinstance(module, nn.BatchNorm2d):
            module.eval()

    predictions = []
    with torch.no_grad():
        for _ in range(num_passes):
            pred = model(image)
            predictions.append(pred)

    preds_tensor = torch.stack(predictions)  # Shape: [num_passes, batch_size, ...]
    mean_prediction = preds_tensor.mean(dim=0)
    uncertainty = preds_tensor.var(dim=0)
    return mean_prediction, uncertainty


In [ ]:
import numpy as np

def select_uncertain_samples(unlabeled_images, model, num_passes=20, k=10):
    """
    Select k most uncertain images from unlabeled_images using MC Dropout.
    """
    uncertainty_scores = []
    for image in unlabeled_images:
        _, uncertainty = mc_dropout_inference(model, image, num_passes=num_passes)
        score = uncertainty.mean().item()
        uncertainty_scores.append(score)

    uncertainty_scores = np.array(uncertainty_scores)
    selected_indices = np.argsort(uncertainty_scores)[-k:]  # top-k uncertain
    selected_images = [unlabeled_images[i] for i in selected_indices]

    return selected_images, uncertainty_scores, selected_indices

In [ ]:
# Initialize dataset
setup_active_dataset()
all_samples = get_all_samples()

# Prepare CSV logging
log_path = "accuracy_log.csv"
with open(log_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Phase", "Num_Images", "Top-1 Accuracy", "Top-5 Accuracy"])

# Initial sampling and train
current_sample_list = stratified_sample(all_samples, N_INITIAL)
copy_samples(current_sample_list)

print("Initial Epoch")
model = YOLO('yolo11n-cls.pt')
results = model.train(data=ACTIVE_DATASET_DIR, epochs=1, imgsz=640, name="active-phase0", project="runs/classify")

# Log initial accuracy
acc1 = results.top1
acc5 = results.top5

with open(log_path, "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([0, len(current_sample_list), acc1, acc5])

# Compute remaining samples
remaining_samples = list(set(all_samples) - set(current_sample_list))

# Active Learning (Uncertainty Sampling)
for phase in range(1, NUM_PHASES):
    n_to_add = min(N_PER_PHASE, len(remaining_samples))
    if n_to_add <= 0:
        print("No more samples")
        break

    print(f"Phase {phase}: selecting top {n_to_add} uncertain samples...")

    # Load model from previous phase
    model = YOLO(f"runs/classify/active-phase{phase-1}/weights/last.pt")

    # Apply uncertainty sampling using MC Dropout
    model.model.train()  # Enable dropout
    for module in model.model.modules():
        if isinstance(module, nn.BatchNorm2d):
            module.eval()

    selected_images, uncertainty_scores, selected_indices = select_uncertain_samples(
        remaining_samples,
        model.model,
        num_passes=20,
        k=n_to_add
    )

    new_samples = [remaining_samples[i] for i in selected_indices]
    copy_samples(new_samples)

    current_sample_list.extend(new_samples)
    remaining_samples = list(set(remaining_samples) - set(new_samples))

    print(f"Phase {phase+1}: add {n_to_add} samples, total sample number: {len(current_sample_list)}")

    # Train model for this phase
    results = model.train(data=ACTIVE_DATASET_DIR, epochs=1, imgsz=640, name=f"active-phase{phase}", project="runs/classify")

    # Log accuracy
    acc1 = results.top1
    acc5 = results.top5

    with open(log_path, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([phase, len(current_sample_list), acc1, acc5])

    print(f"Phase {phase+1} training completed")
